In [103]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

You have to manually download the data for WUO3UK first for people in Scotland that commute somewhere and then for people in England and and Wales that commute to Scotland. This data is at an almost Lad level but needs to be slightly manually altered and it is an approximation. Unfortunately Scottish commuting data doesn't not exist at the MSOA level in an age distributed format.

In [104]:
#Scottish commuting
age16=pd.read_csv("16.csv",sep=',')
age25=pd.read_csv("25.csv",sep=',')
age35=pd.read_csv("35.csv",sep=',')
age50=pd.read_csv("50.csv",sep=',')
age65=pd.read_csv("65.csv",sep=',')
age75=pd.read_csv("75.csv",sep=',')
work_age_groups = ['0-15', '16-24', '25-34', '35-49', '50-64', '65-74', '75+']

In [105]:
names=['index',	'Area of residence','Area of workplace','All categories: Age 16 and over','16-24','25-34','35-49','50-64','65-74','75+']

In [106]:
#Put into the usual form
ScotlandCommuting=pd.DataFrame(columns=names)
SEWCommuting = pd.DataFrame(columns=names)
k=0
k2=0
a=np.zeros(10)
l=np.array(a,dtype=object)
for i in range(0,len(age16)):
    a16=age16.iloc[i][1:len(age16.iloc[i])].values.astype(int)
    a25=age25.iloc[i][1:len(age25.iloc[i])].values.astype(int)
    a35=age35.iloc[i][1:len(age35.iloc[i])].values.astype(int)
    a50=age50.iloc[i][1:len(age50.iloc[i])].values.astype(int)
    a65=age65.iloc[i][1:len(age65.iloc[i])].values.astype(int)
    a75=age75.iloc[i][1:len(age75.iloc[i])].values.astype(int)
    sum1=a16+a25+a35+a50+a65+a75
    for j in range(0,len(sum1)):
        if(sum1[j]>10):
            l[0]=0
            l[1]=age16.columns[j+1]
            l[2]=age16.iloc[i][0]
            l[3]=sum1[j]
            l[4]=a16[j]
            l[5]=a25[j]
            l[6]=a35[j]
            l[7]=a50[j]
            l[8]=a65[j]
            l[9]=a75[j]
            if(np.sum(age16.columns.isin([age16.iloc[i][0]]))>=1):
                ScotlandCommuting.loc[k]=l
                k=k+1
            else:
                SEWCommuting.loc[k2]=l
                k2=k2+1            

In [107]:
#England and Wales commuting to Scotland
age16E=pd.read_csv("EW16.csv",sep=',')
age25E=pd.read_csv("EW25.csv",sep=',')
age35E=pd.read_csv("EW35.csv",sep=',')
age50E=pd.read_csv("EW50.csv",sep=',')
age65E=pd.read_csv("EW65.csv",sep=',')
age75E=pd.read_csv("EW75.csv",sep=',')

In [108]:
#Put into the usual form
EWSCommuting = pd.DataFrame(columns=names)
k=0
a=np.zeros(10)
l=np.array(a,dtype=object)
for i in range(1,len(age16E)):
    a16=age16E.iloc[i][1:len(age16E.iloc[i])].values.astype(int)
    a25=age25E.iloc[i][1:len(age25E.iloc[i])].values.astype(int)
    a35=age35E.iloc[i][1:len(age35E.iloc[i])].values.astype(int)
    a50=age50E.iloc[i][1:len(age50E.iloc[i])].values.astype(int)
    a65=age65E.iloc[i][1:len(age65E.iloc[i])].values.astype(int)
    a75=age75E.iloc[i][1:len(age75E.iloc[i])].values.astype(int)
    sum1=a16+a25+a35+a50+a65+a75
    for j in range(1,len(sum1)):
        if(sum1[j]>20):
            l[0]=0
            l[1]=age25E.iloc[i][0]
            l[2]=age25E.columns[j]
            l[3]=sum1[j]
            l[4]=a16[j]
            l[5]=a25[j]
            l[6]=a35[j]
            l[7]=a50[j]
            l[8]=a65[j]
            l[9]=a75[j]
            EWSCommuting.loc[k]=l
            k=k+1


Then we need to tnafer into proper LAD form for all data. Sometimes maunally and approximately unfortunately.

In [110]:
msoa_to_lad = pd.read_csv('data/PCD_OA_LSOA_MSOA_LAD_FEB20_UK_LU.csv', encoding="latin_1", low_memory=False)

In [111]:
for i in range(0,len(EWSCommuting)):
    b=msoa_to_lad.loc[msoa_to_lad['ladnm']==EWSCommuting['Area of workplace'][i]].reset_index(drop=True)
    if(b.empty!=1):
        EWSCommuting['Area of workplace'][i]=b["ladcd"][0]
    b=msoa_to_lad.loc[msoa_to_lad['ladnm']==EWSCommuting['Area of residence'][i]].reset_index(drop=True)
    if(b.empty!=1):
        EWSCommuting['Area of residence'][i]=b["ladcd"][0]
#because I am too lazy to do this shit manually
    if(i==148):
        EWSCommuting['Area of residence'][i]="W06000014"
    if(EWSCommuting['Area of residence'][i]=='Waveney'):
        EWSCommuting['Area of residence'][i]="E07000244"
    if(EWSCommuting['Area of residence'][i]=='Cornwall,Isles of Scilly'):
        EWSCommuting['Area of residence'][i]="E06000052"

In [96]:
for i in range(0,len(SEWCommuting)):
    b=msoa_to_lad.loc[msoa_to_lad['ladnm']==SEWCommuting['Area of workplace'][i]].reset_index(drop=True)
    if(b.empty!=1):
        SEWCommuting['Area of workplace'][i]=b["ladcd"][0]
    b=msoa_to_lad.loc[msoa_to_lad['ladnm']==SEWCommuting['Area of residence'][i]].reset_index(drop=True)
    if(b.empty!=1):
        SEWCommuting['Area of residence'][i]=b["ladcd"][0]
#same as above
    if(SEWCommuting['Area of workplace'][i]=="Westminster,City of London"):
        SEWCommuting['Area of workplace'][i]="E09000001"

In [97]:
# for i in range(0,len(ScotlandCommuting)):
#     b=msoa_to_lad.loc[msoa_to_lad['ladnm']==ScotlandCommuting['Area of workplace'][i]].reset_index(drop=True)
#     if(b.empty!=1):
#         ScotlandCommuting['Area of workplace'][i]=b["ladcd"][0]
#     b=msoa_to_lad.loc[msoa_to_lad['ladnm']==ScotlandCommuting['Area of residence'][i]].reset_index(drop=True)
#     if(b.empty!=1):
#         ScotlandCommuting['Area of residence'][i]=b["ladcd"][0]
# #same as above
#     if(ScotlandCommuting['Area of residence'][i]=='Eilean Siar'):
#         ScotlandCommuting['Area of residence'][i]='S12000013'
#     if(ScotlandCommuting['Area of workplace'][i]=='Eilean Siar'):
#         ScotlandCommuting['Area of workplace'][i]='S12000013'

In [121]:
#Due to recoding
for i in range(0,len(ScotlandCommuting)):
    if(ScotlandCommuting['Area of residence'][i]=='S12000015'):
        ScotlandCommuting['Area of residence'][i]='S12000047'
    if(ScotlandCommuting['Area of workplace'][i]=='S12000015'):
        ScotlandCommuting['Area of workplace'][i]='S12000047'
    if(ScotlandCommuting['Area of residence'][i]=='S12000024'):
        ScotlandCommuting['Area of residence'][i]='S12000048'
    if(ScotlandCommuting['Area of workplace'][i]=='S12000024'):
        ScotlandCommuting['Area of workplace'][i]='S12000048'
    if(ScotlandCommuting['Area of residence'][i]=='S12000044'):
        ScotlandCommuting['Area of residence'][i]='S12000050'
    if(ScotlandCommuting['Area of workplace'][i]=='S12000044'):
        ScotlandCommuting['Area of workplace'][i]='S12000050'
    if(ScotlandCommuting['Area of residence'][i]=='S12000046'):
        ScotlandCommuting['Area of residence'][i]='S12000049'
    if(ScotlandCommuting['Area of workplace'][i]=='S12000046'):
        ScotlandCommuting['Area of workplace'][i]='S12000049'

Next we divide the English data into MSOAs because that is the level we work at there

In [122]:
SEWCommutingNew = pd.DataFrame(columns=names)
k=0
for i in range(0,len(SEWCommuting)):
    b=msoa_to_lad.loc[msoa_to_lad['ladcd']==SEWCommuting['Area of workplace'][i]].reset_index(drop=True)
    v=b['msoa11cd'].drop_duplicates()
    if(SEWCommuting['All categories: Age 16 and over'][i]>len(v)):
        for j in range(0,len(v)):
            l[0]=0
            l[1]=SEWCommuting.iloc[i][1]
            l[2]=v.iloc[j]
            l[3]=SEWCommuting.iloc[i][3]/len(v)
            l[4]=SEWCommuting.iloc[i][4]/len(v)
            l[5]=SEWCommuting.iloc[i][5]/len(v)
            l[6]=SEWCommuting.iloc[i][6]/len(v)
            l[7]=SEWCommuting.iloc[i][7]/len(v)
            l[8]=SEWCommuting.iloc[i][8]/len(v)
            l[9]=SEWCommuting.iloc[i][9]/len(v)
            SEWCommutingNew.loc[k]=l
            k=k+1

In [123]:
EWSCommutingNew = pd.DataFrame(columns=names)
k=0
for i in range(0,len(EWSCommuting)):
    b=msoa_to_lad.loc[msoa_to_lad['ladcd']==EWSCommuting['Area of residence'][i]].reset_index(drop=True)
    v=b['msoa11cd'].drop_duplicates()
    if(EWSCommuting['All categories: Age 16 and over'][i]>len(v)):
        for j in range(0,len(v)):
            l[0]=0
            l[1]=v.iloc[j]
            l[2]=EWSCommuting.iloc[i][2]
            l[3]=EWSCommuting.iloc[i][3]/len(v)
            l[4]=EWSCommuting.iloc[i][4]/len(v)
            l[5]=EWSCommuting.iloc[i][5]/len(v)
            l[6]=EWSCommuting.iloc[i][6]/len(v)
            l[7]=EWSCommuting.iloc[i][7]/len(v)
            l[8]=EWSCommuting.iloc[i][8]/len(v)
            l[9]=EWSCommuting.iloc[i][9]/len(v)
            EWSCommutingNew.loc[k]=l
            k=k+1

In [124]:
Everything=pd.concat([EWSCommutingNew, SEWCommutingNew, ScotlandCommuting], ignore_index=True)
Everything.to_csv("ScottishCommute.csv",index=False,)

In [120]:
Everything

,index,Area of residence,Area of workplace,All categories: Age 16 and over,16-24,25-34,35-49,50-64,65-74,75+
0,0,E02002498,S12000030,6.05263,0.210526,1.63158,2.52632,1.63158,0.0526316,0
1,0,E02002496,S12000030,6.05263,0.210526,1.63158,2.52632,1.63158,0.0526316,0
2,0,E02002500,S12000030,6.05263,0.210526,1.63158,2.52632,1.63158,0.0526316,0
3,0,E02002504,S12000030,6.05263,0.210526,1.63158,2.52632,1.63158,0.0526316,0
4,0,E02002503,S12000030,6.05263,0.210526,1.63158,2.52632,1.63158,0.0526316,0
...,...,...,...,...,...,...,...,...,...,...
2769,0,S12000041,S12000046,59,8,8,26,15,2,0
2770,0,S12000042,S12000046,84,20,21,25,17,1,0
2771,0,S12000044,S12000046,28034,4374,6890,10700,5723,291,56
2772,0,S12000045,S12000046,21116,2900,3082,8225,6385,442,82
